In [32]:
import tkinter as tk
import customtkinter as ctk
import sqlite3
from tkinter import messagebox as mb
from tkinter import ttk as ttk

dbconnector = sqlite3.connect('books_database.db')
dbcursor = dbconnector.cursor()
dbconnector.execute("""CREATE TABLE IF NOT EXISTS allbooks(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
Title TEXT NOT NULL,
Author TEXT NOT NULL,
Status TEXT NOT NULL)"""
)
dbconnector.commit()

dbconnector.execute("""CREATE TABLE IF NOT EXISTS issuedbooks(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
Title TEXT NOT NULL,
Author TEXT NOT NULL,
Issuedto TEXT NOT NULL,
Status TEXT NOT NULL)"""
)
dbconnector.commit()

def add_book():
    def confirm_clicked():
        # Get the selected status value
        selected_status = status.get()

        # Check if title or author is empty, or if status is not selected
        if not title.get().strip():  
            mb.showerror("Input Error", "Please Enter the Book Title.")
            return
        if not author.get().strip():  
            mb.showerror("Input Error", "Please Enter the Author Name.")
            return
        if selected_status == "Choose Status" or not selected_status:  # Ensure valid status is selected
            mb.showerror("Input Error", "Please Select a Valid Status (Available/Borrowed).")
            return
        confirmation = mb.askyesno("Confirmation", "Are You Sure You Want To Save This Book?")
        if confirmation:
            try:
                dbconnector.execute(
                """INSERT INTO allbooks(Title, Author, Status) VALUES (?,?,?)""",
                (title.get(), author.get(), selected_status)
                )
                dbconnector.commit()
                if status.get() == "Borrowed":
                    dbconnector.execute(
                    """INSERT INTO issuedbooks(Title, Author, Issuedto, Status) VALUES (?,?,?,?)""", (title.get(), author.get(),'N/A', status.get())
                    )
                    dbconnector.commit()
                mb.showinfo("Done", "The Book Has Been Added To the Records Successfully!")
            except Exception as e:
                mb.showerror("Error", f"An error occurred while adding the book: {e}")
            title.set("")
            author.set("")
            status.set("Choose Status")
            addwindow.destroy()
        else:
            addwindow.destroy()

    addwindow = tk.Toplevel()
    addwindow.title("Library Management System")
    addwindow.geometry("600x400+330+50")

    add_frame = ctk.CTkFrame(master=addwindow, fg_color="#FFEFC8")
    add_frame.pack(anchor="center", fill="both", expand="yes")
    add_label_frame = ctk.CTkFrame(master=add_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    add_label_frame.pack(side="top", fill="both", padx=20, pady=30)
    add_label = ctk.CTkLabel(master=add_label_frame, text=" Add Book ",  fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    add_label.pack(side="top", anchor="center", padx=20, pady=20)

    add_entry_frame = ctk.CTkFrame(master=add_frame,fg_color="#E5989B", corner_radius= 15 , border_width=2, border_color="black")
    add_entry_frame.pack(anchor="center", padx=20, pady=10, fill="both")
    add_entry_frame.columnconfigure(0, weight=1)
    add_entry_frame.columnconfigure(1, weight=1)
    add_entry_frame.columnconfigure(2, weight=1)

    # Title Entry
    title_label = ctk.CTkLabel(master=add_entry_frame, text="Title:",text_color="black",font= ('Open Sans', 17))
    title_label.grid(row=1, column=0, sticky="ew", padx=20, pady=10)
    title = tk.StringVar()
    title_entry = ctk.CTkEntry(master=add_entry_frame, textvariable=title, width=200,fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black")
    title_entry.grid(row=1, column=2, padx=20, pady=10)

    # Author Entry
    author_label = ctk.CTkLabel(master=add_entry_frame, text="Author:",text_color="black",font= ('Open Sans', 17))
    author_label.grid(row=2, column=0, sticky="ew", padx=20, pady=10)
    author = tk.StringVar()
    author_entry = ctk.CTkEntry(master=add_entry_frame, textvariable=author, width=200,fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black")
    author_entry.grid(row=2, column=2, padx=20, pady=10)

    # Status ComboBox
    status_label = ctk.CTkLabel(master=add_entry_frame, text="Status:",text_color="black",font= ('Open Sans', 17))
    status_label.grid(row=3, column=0, sticky="ew", padx=20, pady=10)
    status = ctk.StringVar()
    status_entry = ctk.CTkComboBox(
        master=add_entry_frame,
        values=["Choose Status", "Available", "Borrowed"],
        variable=status,
        width=160,fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black",
        button_color="#FFEFC8", dropdown_fg_color="#A594F9", dropdown_font=('Open Sans', 17), dropdown_text_color="black", dropdown_hover_color="#684cf5"
    )
    status_entry.grid(row=3, column=2, padx=20, pady=10)
    status_entry.set("Choose Status") 

    # Add buttons
    add_button_frame = ctk.CTkFrame(master=add_frame,fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    add_button_frame.pack(anchor="center", padx=20, pady=30, fill="both")
    add_button_frame.columnconfigure(0, weight=1)
    add_button_frame.columnconfigure(1, weight=1)
    add_button_frame.columnconfigure(2, weight=1)

    add_confirm_button = ctk.CTkButton(master=add_button_frame, text="Confirm", command=confirm_clicked,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    add_confirm_button.grid(row=0, column=0, padx=20, pady=10)

    add_quit_button = ctk.CTkButton(master=add_button_frame, text="Cancel", command=addwindow.destroy,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    add_quit_button.grid(row=0, column=2, padx=20, pady=10)


def issue_book():
    def confirm_clicked():
    
        if not title.get().strip():
           mb.showerror("Input Error", "Please Enter the Book Title.")
           return
        if not author.get().strip():
            mb.showerror("Input Error", "Please Enter the Author Name.")
            return
        if not issuedto.get().strip():  # Issued To field cannot be empty
            mb.showerror("Input Error", "Please Enter the Name of the Person the Book Is Issued To.")
            return
        confirmation = mb.askyesno("Confirmation", "Are You Sure You Want To Issue This Book?")
        if confirmation:
           status = tk.StringVar()
           status.set("Borrowed")
           try:
                dbconnector.execute(
                """INSERT INTO issuedbooks(Title, Author, Issuedto, Status) VALUES (?,?,?,?)""",
                (title.get(), author.get(),issuedto.get(), status.get())
                )
                dbconnector.commit()
                dbconnector.execute(
                """UPDATE allbooks SET Status = "Borrowed" WHERE LOWER(Title) = LOWER(?) AND LOWER(Author) = LOWER(?)""",
                (title.get(), author.get())
                )
                dbconnector.commit()
                mb.showinfo("Done", "The Book Has Been Added To the Records Successfully!")
           except Exception as e:
                mb.showerror("Error", f"An error occurred while issuing the book: {e}")
           

           title.set("")
           author.set("")
           issuedto.set("")
           issuewindow.destroy()
        else:
            issuewindow.destroy()

    issuewindow = tk.Toplevel()
    issuewindow.title("Library Management System")
    issuewindow.geometry("600x400+330+50")

    issue_frame = ctk.CTkFrame(master=issuewindow,fg_color = "#FFEFC8")
    issue_frame.pack(anchor = "center", fill = "both", expand = "yes")
    issue_label_frame = ctk.CTkFrame(master= issue_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    issue_label_frame.pack(side = "top",  fill = "both", padx = 20, pady = 30)
    issue_label = ctk.CTkLabel(master=issue_label_frame, text=" Issue Book ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    issue_label.pack(side = "top", anchor = "center", padx = 20, pady = 20)
    
    issue_entry_frame = ctk.CTkFrame(master= issue_frame,fg_color="#E5989B", corner_radius= 15 , border_width=2, border_color="black")
    issue_entry_frame.pack(anchor = "center", padx = 20, pady = 10, fill = "both")
    issue_entry_frame.columnconfigure(0, weight=1)
    issue_entry_frame.columnconfigure(1, weight=1)
    issue_entry_frame.columnconfigure(2, weight=1)
    
    title_label =ctk.CTkLabel(master=issue_entry_frame,
    text= "Title:", text_color="black",font= ('Open Sans', 17)
    )
    title_label.grid(row = 1, column = 0, sticky = "ew", padx = 20, pady = 10)
    title = ctk.StringVar()
    title_entry = ctk.CTkEntry(master=issue_entry_frame,
    textvariable = title,
    width = 200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black")
    title_entry.grid(row = 1, column = 2, padx = 20, pady = 10)

    author_label =ctk.CTkLabel(master=issue_entry_frame,
    text= "Author:", text_color="black",font= ('Open Sans', 17)
    )
    author_label.grid(row = 2, column = 0, sticky = "ew", padx = 20, pady = 10)
    author = ctk.StringVar()
    author_entry = ctk.CTkEntry(master=issue_entry_frame,
    textvariable = author,
    width = 200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black")
    author_entry.grid(row = 2, column = 2, padx = 20, pady = 10)

    issuedto_label =ctk.CTkLabel(master=issue_entry_frame,
    text= "Issued To:", text_color="black",font= ('Open Sans', 17)
    )
    issuedto_label.grid(row = 3, column = 0, sticky = "ew", padx = 20, pady = 10)
    issuedto = ctk.StringVar()
    issuedto_entry = ctk.CTkEntry(master=issue_entry_frame,
    textvariable = issuedto,
    width = 200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black")
    issuedto_entry.grid(row = 3, column = 2, padx = 20, pady = 10)

    issue_button_frame = ctk.CTkFrame(master= issue_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    issue_button_frame.pack(anchor = "center", padx = 20, pady = 30, fill = "both")
    issue_button_frame.columnconfigure(0, weight=1)
    issue_button_frame.columnconfigure(1, weight=1)
    issue_button_frame.columnconfigure(2, weight=1)
    add_confirm_button = ctk.CTkButton(master=issue_button_frame, text="Confirm", command= confirm_clicked,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    add_confirm_button.grid(row = 0, column = 0,padx = 20, pady = 10)
    add_quit_button = ctk.CTkButton(master=issue_button_frame, text="Cancel", command=issuewindow.destroy,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    add_quit_button.grid(row = 0, column = 2,padx = 20, pady = 10)



def edit_book():
    def edit_clicked():
      if not title.get().strip():  # Title cannot be empty
        mb.showerror("Input Error", "Please Enter the Book Title.")
        return
      if not author.get().strip():  # Author cannot be empty
        mb.showerror("Input Error", "Please Enter the Author Name.")
        return
      if not status.get() or status.get() == "Choose Status":  # Ensure a valid status is selected
        mb.showerror("Input Error", "Please Select a Book Status.")
        return
      confirmation = mb.askyesno("Confirmation", "Are You Sure You Want To Commit Changes In the Book Information?")
      if confirmation:
        try:
            dbconnector.execute(
            'UPDATE allbooks SET Title = ?, Author = ?, Status = ? WHERE ID = ?',
            (title.get(), author.get(), status.get(), ID.get())
            )
            dbconnector.commit()
            if status.get() == "Borrowed":
             dbconnector.execute("""UPDATE issuedbooks SET Title = ?, Author = ?, Issuedto = ? WHERE ID =?""", 
             title.get(), author.get(), issuedto.get(), ID.get())
             dbconnector.commit()
             return
            mb.showinfo("Done", "Book Information Has Been Edited Successfully!")
        except Exception as e:
                mb.showerror("Error", f"An error occurred while editing the book: {e}")
      
        ID.set("")
        title.set("")
        author.set("")
        status.set("")
        issuedto.set("")
        edit_window.destroy()
      else:
        edit_window.destroy()

    
    edit_window = tk.Toplevel()
    edit_window.title("Library Management System")
    edit_window.geometry("600x482+330+50")
  
    edit_frame = ctk.CTkFrame(master=edit_window, fg_color = "#FFEFC8")
    edit_frame.pack(anchor = "center", fill = "both", expand = "yes")
    edit_label_frame = ctk.CTkFrame(master= edit_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    edit_label_frame.pack(side = "top",  fill = "both", padx = 20, pady = 30)
    edit_label = ctk.CTkLabel(master=edit_label_frame, text=" Edit Book Information ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    edit_label.pack(side = "top", anchor = "center", padx = 20, pady = 10)
    
    edit_entry_frame = ctk.CTkFrame(master= edit_frame, fg_color="#E5989B", corner_radius= 15 , border_width=2, border_color="black")
    edit_entry_frame.pack(anchor = "center", padx = 20, pady = 10, fill = "both")
    edit_entry_frame.columnconfigure(0, weight=1)
    edit_entry_frame.columnconfigure(1, weight=1)
    edit_entry_frame.columnconfigure(2, weight=1)

    ID_label =ctk.CTkLabel(master=edit_entry_frame,
    text= "ID in 'All Books':", text_color="black",font= ('Open Sans', 17)
    )
    ID_label.grid(row = 1, column = 0, sticky = "ew", padx = 20, pady = 10)
    ID = ctk.StringVar()
    ID_entry = ctk.CTkEntry(master=edit_entry_frame,
    textvariable= ID,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    ID_entry.grid(row = 1, column = 2, padx = 20, pady = 10)


    title_label =ctk.CTkLabel(master=edit_entry_frame,
    text= "Title:", text_color="black",font= ('Open Sans', 17)
    )
    title_label.grid(row = 2, column = 0, sticky = "ew", padx = 20, pady = 10)
    title = ctk.StringVar()
    title_entry = ctk.CTkEntry(master=edit_entry_frame,
    textvariable= title,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    title_entry.grid(row = 2, column = 2, padx = 20, pady = 10)

    author_label =ctk.CTkLabel(master=edit_entry_frame,
    text= "Author:", text_color="black",font= ('Open Sans', 17)
    )
    author_label.grid(row = 3, column = 0, sticky = "ew", padx = 20, pady = 10)
    author = ctk.StringVar()
    author_entry = ctk.CTkEntry(master=edit_entry_frame,
    textvariable= author,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    author_entry.grid(row = 3, column = 2, padx = 20, pady = 10)

    status_label =ctk.CTkLabel(master=edit_entry_frame,
    text= "Status:", text_color="black",font= ('Open Sans', 17)
    )
    status_label.grid(row = 4, column = 0, sticky = "ew", padx = 20, pady = 10)
    status = ctk.StringVar()
    status_entry = ctk.CTkComboBox(master=edit_entry_frame,
        values=["Choose Status", "Available", "Borrowed"],
        variable= status,
        width=160, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black",
        button_color="#FFEFC8", dropdown_fg_color="#A594F9", dropdown_font=('Open Sans', 17), dropdown_text_color="black", dropdown_hover_color="#684cf5")
    status_entry.set("Choose Status")
    status_entry.grid(row = 4, column = 2, padx = 20, pady = 10)

    issuedto_label =ctk.CTkLabel(master=edit_entry_frame,
    text= "Issued To:", text_color="black",font= ('Open Sans', 17)
    )
    issuedto_label.grid(row = 5, column = 0, sticky = "ew", padx = 20, pady = 10)
    issuedto = ctk.StringVar()
    issuedto_entry = ctk.CTkEntry(master=edit_entry_frame,
    textvariable= issuedto,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    issuedto_entry.grid(row = 5, column = 2, padx = 20, pady = 10)

    edit_button_frame = ctk.CTkFrame(master= edit_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    edit_button_frame.pack(anchor = "center", padx = 20, pady = 30, fill = "both")
    edit_button_frame.columnconfigure(0, weight=1)
    edit_button_frame.columnconfigure(1, weight=1)
    edit_button_frame.columnconfigure(2, weight=1)
    edit_confirm_button = ctk.CTkButton(master=edit_button_frame, text="Confirm", command=edit_clicked,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    edit_confirm_button.grid(row = 0, column = 0,padx = 20, pady = 10)
    edit_quit_button = ctk.CTkButton(master=edit_button_frame, text="Cancel", command= edit_window.destroy,
        fg_color="#A594F9", text_color="black", corner_radius= 12, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    edit_quit_button.grid(row = 0, column = 2,padx = 20, pady = 10)


def delete_book():
    def remove_book():
        if not title.get().strip(): 
           mb.showerror("Input Error", "Please Enter the Book Title.")
           return
        book_info = dbconnector.execute("""SELECT 1 FROM allbooks WHERE LOWER(Title) = LOWER(?) LIMIT 1""", (title.get(),))
        book_exists = book_info.fetchone()
        if not book_exists:
            mb.showinfo("Error", "No Book Found with the Given Title.")
            return
        confirmation = mb.askyesno("Confirmation", "Are You Sure You Want To Delete This Book From Records?")
        if confirmation:
            try:
                dbconnector.execute("""DELETE FROM allbooks WHERE LOWER(Title) = LOWER(?)""",(title.get(),))
                dbconnector.commit()
                dbconnector.execute("""DELETE FROM issuedbooks WHERE LOWER(Title) = LOWER(?)""", (title.get(),))
                dbconnector.commit()
                mb.showinfo('Success',"Book Record Deleted Successfully")
            except Exception as e:
                mb.showerror("Error", f"An error occurred while deleting the book: {e}")
            title.set("")
            delete_window.destroy()
    
    delete_window = tk.Toplevel()
    delete_window.title("Library Management System")
    delete_window.geometry("400x287+330+50")

    issuedbook_frame = ctk.CTkFrame(master=delete_window, fg_color="#FFEFC8")
    issuedbook_frame.pack(anchor = "center", fill = "both", expand = "yes")

    label_frame = ctk.CTkFrame(master=issuedbook_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    label_frame.pack(side = "top", anchor = "center", fill = "both", padx = 20, pady = 30)
    label_upper = ctk.CTkLabel(master=label_frame, text=" Delete Book ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    label_upper.pack(anchor = "center", side = "top",padx = 20, pady = 10)

    delete_entry_frame = ctk.CTkFrame(master= issuedbook_frame, fg_color="#E5989B", corner_radius= 15 , border_width=2, border_color="black")
    delete_entry_frame.pack(anchor = "center", padx = 20, pady = 10, fill = "both")
    delete_entry_frame.columnconfigure(0, weight=1)
    delete_entry_frame.columnconfigure(1, weight=1)
    delete_entry_frame.columnconfigure(2, weight=1)

    title_label =ctk.CTkLabel(master=delete_entry_frame,
    text= "Title:", text_color="black",font= ('Open Sans', 17)
    )
    title_label.grid(row = 1, column = 0, sticky = "ew", padx = 20, pady = 10)
    title = tk.StringVar()
    title_entry = ctk.CTkEntry(master=delete_entry_frame,
    textvariable= title,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    title_entry.grid(row = 1, column = 2, padx = 20, pady = 10)

    delete_button_frame = ctk.CTkFrame(master= issuedbook_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    delete_button_frame.pack(anchor = "center", padx = 20, pady = 30, fill = "both")
    delete_button_frame.columnconfigure(0, weight=1)
    delete_button_frame.columnconfigure(1, weight=1)
    delete_button_frame.columnconfigure(2, weight=1)

    delete_confirm_button = ctk.CTkButton(master=delete_button_frame, text="Confirm", command=remove_book,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    delete_confirm_button.grid(row = 0, column = 0,padx = 20, pady = 10)
    delete_quit_button = ctk.CTkButton(master=delete_button_frame, text="Cancel", command= delete_window.destroy,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    delete_quit_button.grid(row = 0, column = 2,padx = 20, pady = 10)

def view_books():
    def display_books():
        book_table.delete(*book_table.get_children())
        data = dbconnector.execute('SELECT * FROM allbooks')
        dbconnector.commit()
        # getting the data in the form of a tuple using fetchall method and storing it in variable "d"
        d = data.fetchall()
        for values in d:
           book_table.insert("","end", values=values)


    allbookswindow = tk.Toplevel()
    allbookswindow.title("Library Management System")
    allbookswindow.geometry("700x600+330+50")

    allmain_frame = ctk.CTkFrame(master=allbookswindow, fg_color="#92C7CF")
    allmain_frame.pack(anchor = "center", fill = "both", expand = True)

    label_frame = ctk.CTkFrame(master=allmain_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    label_frame.pack(side = "top", anchor = "center", fill = "both", padx = 20, pady = 20)
    label_upper = ctk.CTkLabel(master=label_frame, text=" Books in Library ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    label_upper.pack(anchor = "center", side = "top", padx = 20, pady = 10)

    button_frame = ctk.CTkFrame(master=allmain_frame, fg_color="#4C7B8B", border_width=2, border_color="black", corner_radius= 15)
    button_frame.rowconfigure(0, weight=1)
    button_frame.rowconfigure(1, weight=1)
    button_frame.rowconfigure(2, weight=1)
    button_frame.columnconfigure(0, weight=1)
    button_frame.columnconfigure(1, weight=1)
    button_frame.columnconfigure(2, weight=1)
    button_frame.pack(side = "top", fill = "both", padx = 20, pady = 10)

    refresh_button = ctk.CTkButton(master=button_frame, text="Refresh List", command= display_books,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    refresh_button.grid(row = 1, column = 1, padx = 20, pady = 10)

    style = ttk.Style()
    style.configure("Custom.Treeview",
                background="#92C7CF", 
                foreground="black", 
                fieldbackground="#92C7CF", 
                font=("Open Sans", 11))
    style.map("Custom.Treeview",
                background=[('selected', '#E5989B')],
                foreground=[('selected', 'black')])
    style.configure("Custom.Treeview.Heading",
                font=("Open Sans", 12), 
                foreground="black")

    table_frame = ctk.CTkFrame(master=allmain_frame, fg_color="#A594F9", corner_radius= 15 , border_width=2, border_color="black")
    table_frame.pack(anchor = "center", fill = "both", expand = "yes", padx = 20, pady = 20)
    book_table = ttk.Treeview(table_frame, selectmode="browse", columns=('ID', 'Title', 'Author', 'Status'), style= "Custom.Treeview")
    book_table.heading('ID', text='ID', anchor="center")
    book_table.heading('Title', text='Title', anchor="center")
    book_table.heading('Author', text='Author', anchor="center")
    book_table.heading("Status", text="Status", anchor="center")
    book_table.column('#0', width=0, stretch="no")
    book_table.column('#1', width=70, stretch="yes")
    book_table.column('#2', width=100, stretch="yes")
    book_table.column('#3', width=100, stretch="yes")
    book_table.column('#4', width=110, stretch="yes")
    book_table.pack(fill="both", expand="yes", padx=20, pady=20)

    y_scrollbar = ctk.CTkScrollbar(book_table, orientation="vertical", command=book_table.yview, fg_color="#A594F9", button_color="#4C7B8B", button_hover_color="#2F4F56")
    y_scrollbar.pack(side="right", fill="y")
    x_scrollbar = ctk.CTkScrollbar(book_table, orientation="horizontal", command=book_table.xview, fg_color="#A594F9", button_color="#4C7B8B", button_hover_color="#2F4F56")
    x_scrollbar.pack(side="bottom", fill="x")
    display_books()


def return_book():
    def confirm_clicked():
        if not title.get().strip(): 
           mb.showerror("Input Error", "Please Enter the Book Title.")
           return
        book_info = dbconnector.execute("""SELECT 1 FROM allbooks WHERE LOWER(Title) = LOWER(?) LIMIT 1""", (title.get(),))
        book_exists = book_info.fetchone()
        if not book_exists:
            mb.showinfo("Error", "No Book Found with the Given Title.")
            return
        confirmation = mb.askyesno("Confirmation", "Are You Sure You Want To Return This Book?")
        if confirmation:
            try:
                dbconnector.execute("""DELETE FROM issuedbooks WHERE LOWER(Title) = LOWER(?)""",(title.get(),))
                dbconnector.commit()
                dbconnector.execute("""UPDATE allbooks SET Status = ? WHERE LOWER(Title) = LOWER(?)""", ('Available', title.get()))
                dbconnector.commit()
                mb.showinfo('Success',"Book Returned Successfully!")
            except Exception as e:
                mb.showerror("Error", f"An error occurred while returning the book: {e}")
            title.set("")
            return_window.destroy()
        else:
            return_window.destroy()

    return_window = tk.Toplevel()
    return_window.title("Library Management System")
    return_window.geometry("400x287+330+50")

    issuedbook_frame = ctk.CTkFrame(master=return_window, fg_color="#FFEFC8")
    issuedbook_frame.pack(anchor = "center", fill = "both", expand = "yes")

    label_frame = ctk.CTkFrame(master=issuedbook_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    label_frame.pack(side = "top", anchor = "center", fill = "both", padx = 20, pady = 30)
    label_upper = ctk.CTkLabel(master=label_frame, text=" Return Book ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    label_upper.pack(anchor = "center", side = "top",padx = 20, pady = 10)

    return_entry_frame = ctk.CTkFrame(master= issuedbook_frame, fg_color="#E5989B", corner_radius= 15 , border_width=2, border_color="black")
    return_entry_frame.pack(anchor = "center", padx = 20, pady = 10, fill = "both")
    return_entry_frame.columnconfigure(0, weight=1)
    return_entry_frame.columnconfigure(1, weight=1)
    return_entry_frame.columnconfigure(2, weight=1)

    title_label =ctk.CTkLabel(master=return_entry_frame,
    text= "Title:", text_color="black",font= ('Open Sans', 17)
    )
    title_label.grid(row = 1, column = 0, sticky = "ew", padx = 20, pady = 10)
    title = tk.StringVar()
    title_entry = ctk.CTkEntry(master=return_entry_frame,
    textvariable= title,
    width=200, fg_color="#A594F9", text_color="black", corner_radius= 15, font= ('Open Sans', 17),border_width=2, border_color="black"
    )
    title_entry.grid(row = 1, column = 2, padx = 20, pady = 10)

    return_button_frame = ctk.CTkFrame(master= issuedbook_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    return_button_frame.pack(anchor = "center", padx = 20, pady = 30, fill = "both")
    return_button_frame.columnconfigure(0, weight=1)
    return_button_frame.columnconfigure(1, weight=1)
    return_button_frame.columnconfigure(2, weight=1)

    return_confirm_button = ctk.CTkButton(master=return_button_frame, text="Confirm", command=confirm_clicked,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    return_confirm_button.grid(row = 0, column = 0,padx = 20, pady = 10)
    return_quit_button = ctk.CTkButton(master=return_button_frame, text="Cancel", command= return_window.destroy,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    return_quit_button.grid(row = 0, column = 2,padx = 20, pady = 10)

def view_issued():
    def display_issued():
        book_table.delete(*book_table.get_children())
        data = dbconnector.execute('SELECT * FROM issuedbooks')
        dbconnector.commit()
        # getting the data in the form of a tuple using fetchall method and storing it in variable "d"
        d = data.fetchall()
        for values in d:
           book_table.insert("","end", values=values)


    issuedbookswind = tk.Toplevel()
    issuedbookswind.title("Library Management System")
    issuedbookswind.geometry("600x500+330+50")

    issuedbook_frame = ctk.CTkFrame(master=issuedbookswind, fg_color= "#92C7CF")
    issuedbook_frame.pack(anchor = "center", fill = "both", expand = "yes")

    issuedlabel_frame = ctk.CTkFrame(master=issuedbook_frame, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
    issuedlabel_frame.pack(side = "top", anchor = "center", fill = "both", padx = 20, pady = 20)
    issuedlabel_upper = ctk.CTkLabel(master=issuedlabel_frame, text=" Books Borrowed From the Library ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
    issuedlabel_upper.pack(anchor = "center", side = "top", padx = 20, pady = 10)

    issued_button_frame = ctk.CTkFrame(master=issuedbook_frame, fg_color="#4C7B8B", border_width=2, border_color="black", corner_radius= 15)
    issued_button_frame.rowconfigure(0, weight=1)
    issued_button_frame.rowconfigure(1, weight=1)
    issued_button_frame.rowconfigure(2, weight=1)
    issued_button_frame.columnconfigure(0, weight=1)
    issued_button_frame.columnconfigure(1, weight=1)
    issued_button_frame.columnconfigure(2, weight=1)
    issued_button_frame.pack(side = "top", fill = "both", padx = 20, pady = 10)

    issued_refresh_button = ctk.CTkButton(master=issued_button_frame, text="Refresh List", command= display_issued,
        fg_color="#A594F9", text_color="black", corner_radius= 15, hover=True, hover_color="#684cf5", font=('Open Sans', 15), border_width=2, border_color="black")
    issued_refresh_button.grid(row = 1, column = 1,padx = 20, pady = 10)

    style = ttk.Style()
    style.configure("Custom.Treeview",
                background="#92C7CF", 
                foreground="black", 
                fieldbackground="#92C7CF", 
                font=("Open Sans", 11))
    style.map("Custom.Treeview",
                background=[('selected', '#E5989B')],
                foreground=[('selected', 'black')])
    style.configure("Custom.Treeview.Heading",
                font=("Open Sans", 12), 
                foreground="black")

    issued_table_frame = ctk.CTkFrame(master=issuedbook_frame, fg_color="#A594F9", corner_radius= 15 , border_width=2, border_color="black")
    issued_table_frame.pack(anchor = "center", fill = "both", expand = "yes", padx=20, pady=20)
    book_table = ttk.Treeview(issued_table_frame, selectmode="browse", columns=('ID', 'Title', 'Author', 'Issuedto', 'Status'), style= "Custom.Treeview")
    book_table.heading('ID', text='ID', anchor="center")
    book_table.heading('Title', text='Title', anchor="center")
    book_table.heading('Author', text='Author', anchor="center")
    book_table.heading('Issuedto', text='Issued To', anchor="center")
    book_table.heading("Status", text="Status", anchor="center")
    book_table.column('#0', width=0, stretch=False)
    book_table.column('#1', width=70, stretch=True)
    book_table.column('#2', width=100, stretch=True)
    book_table.column('#3', width=100, stretch=True)
    book_table.column('#4', width=110, stretch=True)
    book_table.column('#5', width=110, stretch=True)
    book_table.pack(fill="both", expand="yes", padx=20, pady=20)

    y_scrollbar = ctk.CTkScrollbar(book_table, orientation="vertical", command=book_table.yview, fg_color="#A594F9", button_color="#4C7B8B", button_hover_color="#2F4F56")
    y_scrollbar.pack(side="right", fill="y")
    x_scrollbar = ctk.CTkScrollbar(book_table, orientation="horizontal", command=book_table.xview, fg_color="#A594F9", button_color="#4C7B8B", button_hover_color="#2F4F56")
    x_scrollbar.pack(side="bottom", fill="x")
    display_issued()

mainwindow = ctk.CTk(fg_color="#FFEFC8")
mainwindow.title("Library Management System")
mainwindow.geometry("700x600+330+50")
mainwindow._set_appearance_mode("light")
mainframe = ctk.CTkFrame(master=mainwindow, fg_color="#FFEFC8")
mainframe.pack(anchor = "center", fill = "both", expand = True)
label_frame = ctk.CTkFrame(master= mainframe, fg_color="#F7B5CA",border_width=2, border_color="black", corner_radius= 15)
label_frame.pack(side = "top", anchor = "center", fill = "both", padx = 20, pady = 30)
main_label = ctk.CTkLabel(master=label_frame, text="  Welcome To Library Management System! ", fg_color="#F7B5CA", text_color="black", corner_radius= 15, font= ('Open Sans', 22))
main_label.pack(side = "top", anchor = "center", padx = 20, pady =30)
button_frame = ctk.CTkFrame(master= mainframe,fg_color="#92C7CF",border_width=2, border_color="black", corner_radius= 15)
button_frame.pack(anchor = "center", fill = "both", expand = True, padx = 20, pady = 30)
view_all_books = ctk.CTkButton(master=button_frame, text="View All Books", command= view_books,
                            fg_color="#FFCDB2", text_color="black", corner_radius= 15, hover=True, hover_color="#A6CDC6", font=('Open Sans', 15), border_width=2, border_color="black")
view_all_books.place(relx = 0.5, rely = 0.2, anchor = "center")
add_books = ctk.CTkButton(master=button_frame, text="Add Book",command= add_book,
                            fg_color="#FFB4A2", text_color="black", corner_radius= 15, hover=True, hover_color="#3B6790", font=('Open Sans', 15), border_width=2, border_color="black")
add_books.place(relx = 0.5, rely = 0.3, anchor = "center")
edit_books = ctk.CTkButton(master=button_frame, text="Edit Book", command= edit_book,
                        fg_color="#E5989B", text_color="black", corner_radius= 15, hover=True, hover_color="#4C7B8B", font=('Open Sans', 15) , border_width=2, border_color="black" )
edit_books.place(relx = 0.5, rely = 0.4, anchor = "center")
delete_books = ctk.CTkButton(master=button_frame, text="Delete Book", command= delete_book,
                            fg_color="#EFB6C8", text_color="black", corner_radius= 15, hover=True, hover_color="#C4D9FF", font=('Open Sans', 15), border_width=2, border_color="black" )
delete_books.place(relx = 0.5, rely = 0.5, anchor = "center")
issue_books = ctk.CTkButton(master=button_frame, text="Issue Book", command= issue_book,
                            fg_color="#FFD0D0", text_color="black", corner_radius= 15, hover=True, hover_color="#C5BAFF", font=('Open Sans', 15), border_width=2, border_color="black" )
issue_books.place(relx = 0.5, rely = 0.6, anchor = "center")
return_books = ctk.CTkButton(master=button_frame, text="Return Book", command=return_book,
                            fg_color="#E493B3", text_color="black", corner_radius= 15, hover=True, hover_color="#A594F9", font=('Open Sans', 15),border_width=2, border_color="black")
return_books.place(relx = 0.5, rely = 0.7, anchor = "center")
view_issued_books = ctk.CTkButton(master=button_frame, text="View Issued Books", command=view_issued,
                            fg_color="#E6A4B4", text_color="black", corner_radius= 15, hover=True, hover_color="#92C7CF", font=('Open Sans', 15), border_width=2, border_color="black")
view_issued_books.place(relx = 0.5, rely = 0.8, anchor = "center")
mainwindow.mainloop()